In [1]:
import importlib
from pathlib import Path

import pandas as pd
from rich.console import Console

c = Console(width=60)

GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

# Quick Start

In [4]:
from cuery import Prompt, Response, Task, set_api_keys

set_api_keys()


# Define response model
class Entity(Response):
    name: str
    type: str


class NamedEntities(Response):
    entities: list[Entity]


context = pd.DataFrame(
    {
        "text": [
            "Apple is headquartered in Cupertino, California.",
            "Barack Obama was the 44th President of the United States.",
            "The Eiffel Tower is located in Paris, France.",
        ]
    }
)

prompt = Prompt.from_string("Extract named entities from the following text: {{text}}")
task = Task(prompt=prompt, response=NamedEntities)
result = await task(context, model="openai/gpt-3.5-turbo", n_concurrent=10)
print(result.to_pandas(explode=False))

Gathering responses: 100%|██████████| 3/3 [00:01<00:00,  2.12it/s]

                                                text  \
0   Apple is headquartered in Cupertino, California.   
1  Barack Obama was the 44th President of the Uni...   
2      The Eiffel Tower is located in Paris, France.   

                         entities_name                       entities_type  
0       [Apple, Cupertino, California]  [Organization, Location, Location]  
1  [Barack Obama, 44th, United States]         [Person, Ordinal, Location]  
2        [Eiffel Tower, Paris, France]      [location, location, location]  


In [5]:
print(result.to_pandas(explode=True))

                                                text           name  \
0   Apple is headquartered in Cupertino, California.          Apple   
1   Apple is headquartered in Cupertino, California.      Cupertino   
2   Apple is headquartered in Cupertino, California.     California   
3  Barack Obama was the 44th President of the Uni...   Barack Obama   
4  Barack Obama was the 44th President of the Uni...           44th   
5  Barack Obama was the 44th President of the Uni...  United States   
6      The Eiffel Tower is located in Paris, France.   Eiffel Tower   
7      The Eiffel Tower is located in Paris, France.          Paris   
8      The Eiffel Tower is located in Paris, France.         France   

           type  
0  Organization  
1      Location  
2      Location  
3        Person  
4       Ordinal  
5      Location  
6      location  
7      location  
8      location  


# Prompts

In [6]:
from cuery import Prompt

prompt = Prompt.from_string("Explain {{ topic }} in simple terms.")
c.print(prompt)

╭───────────────────────── PROMPT ─────────────────────────╮
│                                                          │
│  Required: ['topic']                                     │
│                                                          │
│ ╭──────────────────────── USER ────────────────────────╮ │
│ │ Explain {{ topic }} in simple terms.                 │ │
│ ╰──────────────────────────────────────────────────────╯ │
╰──────────────────────────────────────────────────────────╯

# Context

In [7]:
from cuery.context import iter_context

df = pd.DataFrame(
    {
        "topic": ["Machine Learning", "Natural Language Processing", "Computer Vision"],
        "audience": ["beginners", "developers", "researchers"],
    }
)

contexts, count = iter_context(df, required=["topic", "audience"])
next(contexts)

{'topic': 'Machine Learning', 'audience': 'beginners'}

# Response, Task

In [8]:
from cuery import Field, Prompt, Response, Task


class MovieRecommendation(Response):
    title: str
    year: int = Field(gt=1900, lt=2030)
    genre: list[str]
    rating: float = Field(ge=0, le=10)


# Multi-output response model
class MovieRecommendations(Response):
    recommendations: list[MovieRecommendation]


movie_prompt = Prompt.from_string(
    "Recommend a movie for {{ audience }} interested in {{ topic }}."
)
context = [
    {"topic": "Machine Learning", "audience": "beginners"},
    {"topic": "Computer Vision", "audience": "researchers"},
]

movie_task = Task(prompt=movie_prompt, response=MovieRecommendations)
result = await movie_task(context)
print(result)
print(result.to_pandas())

Iterating context:   0%|          | 0/2 [00:00<?, ?it/s]

[MovieRecommendations(recommendations=[MovieRecommendation(title='The Matrix', year=1999, genre=['Action', 'Sci-Fi'], rating=8.7), MovieRecommendation(title='Ex Machina', year=2014, genre=['Drama', 'Sci-Fi'], rating=7.7), MovieRecommendation(title='Her', year=2013, genre=['Drama', 'Romance', 'Sci-Fi'], rating=8.0), MovieRecommendation(title='I, Robot', year=2004, genre=['Action', 'Sci-Fi', 'Thriller'], rating=7.1)]), MovieRecommendations(recommendations=[MovieRecommendation(title='Ex Machina', year=2014, genre=['Drama', 'Sci-Fi', 'Thriller'], rating=7.7), MovieRecommendation(title='Her', year=2013, genre=['Drama', 'Romance', 'Sci-Fi'], rating=8.0), MovieRecommendation(title='Blade Runner 2049', year=2017, genre=['Drama', 'Mystery', 'Sci-Fi'], rating=8.0)])]
              topic     audience              title  year  \
0  Machine Learning    beginners         The Matrix  1999   
1  Machine Learning    beginners         Ex Machina  2014   
2  Machine Learning    beginners                H

In [9]:
print(result.usage())

   prompt  completion      cost
0     147         106  0.000233
1     147          90  0.000208


In [10]:
from cuery import pprint

print(result[0]._raw_response.model)
print(result[0]._raw_response.service_tier)

gpt-3.5-turbo-0125
default


In [12]:
print(movie_task.queries[0]["messages"][0]["content"])

Recommend a movie for beginners interested in Machine Learning.


In [13]:
c.print(result[0])

╭───────────── RESPONSE: MovieRecommendations ─────────────╮
│                                                          │
│ ╭─ recommendations: list[__main__.MovieRecommendation]─╮ │
│ │                                                      │ │
│ │  {'required': True}                                  │ │
│ │                                                      │ │
│ ╰──────────────────────────────────────────────────────╯ │
│                                                          │
│  ╭───────── RESPONSE: MovieRecommendation ──────────╮    │
│  │                                                  │    │
│  │ ╭─ title: str ─────────────────────────────────╮ │    │
│  │ │                                              │ │    │
│  │ │  {'required': True}                          │ │    │
│  │ │                                              │ │    │
│  │ ╰──────────────────────────────────────────────╯ │    │
│  │ ╭─ year: int ──────────────────────────────────╮ │    │
│  │ │                                              │ │    │
│  │ │  {                                           │ │    │
│  │ │      'required': True,                       │ │    │
│  │ │      'metadata': [                           │ │    │
│  │ │          Gt(gt=1900),                        │ │    │
│  │ │          Lt(lt=2030)                         │ │    │
│  │ │      ]                                       │ │    │
│  │ │  }                                           │ │    │
│  │ │                                              │ │    │
│  │ ╰──────────────────────────────────────────────╯ │    │
│  │ ╭─ genre: list[str] ───────────────────────────╮ │    │
│  │ │                                              │ │    │
│  │ │  {'required': True}                          │ │    │
│  │ │                                              │ │    │
│  │ ╰──────────────────────────────────────────────╯ │    │
│  │ ╭─ rating: float ──────────────────────────────╮ │    │
│  │ │                                              │ │    │
│  │ │  {                                           │ │    │
│  │ │      'required': True,                       │ │    │
│  │ │      'metadata': [Ge(ge=0), Le(le=10)]       │ │    │
│  │ │  }                                           │ │    │
│  │ │                                              │ │    │
│  │ ╰──────────────────────────────────────────────╯ │    │
│  │                                                  │    │
│  ╰──────────────────────────────────────────────────╯    │
│                                                          │
│                                                          │
╰──────────────────────────────────────────────────────────╯

# Chain

In [15]:
from typing import Literal

from cuery import Chain, Task

# Reuse example from above
movies = Task(prompt=movie_prompt, response=MovieRecommendations)


# Add PG rating
class Rating(Response):
    pg_category: Literal["G", "PG", "PG-13", "R", "NC-17"]
    """PG rating of the movie."""
    pg_reason: str
    """Reason for the rating."""


rating_prompt = Prompt.from_string("What is the PG rating for {{ title }}?")
rating_task = Task(prompt=rating_prompt, response=Rating)

# Create a chain of tasks
chain = Chain(movie_task, rating_task)
result = await chain(context, model="openai/gpt-3.5-turbo", n_concurrent=20)
result

[08/18/25 15:24:36] INFO     [1/2] Running task 'MovieRecommendations'

Gathering responses: 100%|██████████| 2/2 [00:02<00:00,  1.02s/it]


[08/18/25 15:24:38] INFO     [2/2] Running task 'Rating'

Gathering responses: 100%|██████████| 6/6 [00:01<00:00,  4.37it/s]


,topic,audience,title,year,genre,rating,pg_category,pg_reason
0,Machine Learning,beginners,The Imitation Game,2014,"[Biography, Drama, Thriller]",8.0,PG-13,"for some sexual references, mature thematic ma..."
1,Machine Learning,beginners,Ex Machina,2014,"[Drama, Mystery, Sci-Fi]",7.7,R,"Ex Machina contains strong violence, language,..."
2,Machine Learning,beginners,Ex Machina,2014,"[Drama, Mystery, Sci-Fi]",7.7,R,"Ex Machina contains strong language, violence,..."
3,Machine Learning,beginners,Her,2013,"[Drama, Romance, Sci-Fi]",8.0,R,"Language, sexual content and brief graphic nudity"
4,Machine Learning,beginners,Her,2013,"[Drama, Romance, Sci-Fi]",8.0,R,"For language, sexual content and brief nudity."
5,Computer Vision,researchers,Blade Runner,1982,"[Sci-Fi, Thriller]",8.1,PG,violence and brief nudity
6,Computer Vision,researchers,Ex Machina,2014,"[Drama, Sci-Fi]",7.7,R,"Ex Machina contains strong violence, language,..."
7,Computer Vision,researchers,Ex Machina,2014,"[Drama, Sci-Fi]",7.7,R,"Ex Machina contains strong language, violence,..."
8,Computer Vision,researchers,Her,2013,"[Drama, Romance, Sci-Fi]",8.0,R,"Language, sexual content and brief graphic nudity"
9,Computer Vision,researchers,Her,2013,"[Drama, Romance, Sci-Fi]",8.0,R,"For language, sexual content and brief nudity."


In [16]:
print(result)

              topic     audience               title  year  \
0  Machine Learning    beginners  The Imitation Game  2014   
1  Machine Learning    beginners          Ex Machina  2014   
2  Machine Learning    beginners          Ex Machina  2014   
3  Machine Learning    beginners                 Her  2013   
4  Machine Learning    beginners                 Her  2013   
5   Computer Vision  researchers        Blade Runner  1982   
6   Computer Vision  researchers          Ex Machina  2014   
7   Computer Vision  researchers          Ex Machina  2014   
8   Computer Vision  researchers                 Her  2013   
9   Computer Vision  researchers                 Her  2013   

                          genre  rating pg_category  \
0  [Biography, Drama, Thriller]     8.0       PG-13   
1      [Drama, Mystery, Sci-Fi]     7.7           R   
2      [Drama, Mystery, Sci-Fi]     7.7           R   
3      [Drama, Romance, Sci-Fi]     8.0           R   
4      [Drama, Romance, Sci-Fi]     8.0   

# Templates

In [ ]:
from cuery import templates
from cuery.utils import render_template

prompt = """
Analyze the following record and provide a summary in Markdown format.
Make sure to include the name, age, city, and interests.
"""

context = {
    "record": {
        "name": "John Doe",
        "age": 30,
        "city": "New York",
        "interests": ["reading", "traveling", "coding"],
    }
}


In [4]:
for mode in ["json", "text", "md"]:
    tpl = templates.prompt_with_template(prompt, template=f"record_to_{mode}")
    print(render_template(tpl, **context), end="\n\n")


Analyze the following record and provide a summary in Markdown format.
Make sure to include the name, age, city, and interests.

{
  "age": 30,
  "city": "New York",
  "interests": [
    "reading",
    "traveling",
    "coding"
  ],
  "name": "John Doe"
}


Analyze the following record and provide a summary in Markdown format.
Make sure to include the name, age, city, and interests.

name: "John Doe"

age: 30

city: "New York"

interests: [
  "reading",
  "traveling",
  "coding"
]




Analyze the following record and provide a summary in Markdown format.
Make sure to include the name, age, city, and interests.

## name

John Doe


## age

30


## city

New York


## interests

- reading

- traveling

- coding






